In [1]:
!pip install transformers sentencepiece datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 51.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 55.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 36.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 65.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 29.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━

In [2]:
import tensorflow as tf
from transformers import AutoTokenizer, AdamWeightDecay, TFAutoModelForSeq2SeqLM
from transformers.keras_callbacks import KerasMetricCallback
from transformers.keras_callbacks import PushToHubCallback
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
import numpy as np
from nltk.translate.bleu_score import sentence_bleu
from transformers import GPT2Tokenizer, GPT2Model
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast
from transformers import AutoModelWithLMHead, AutoTokenizer, pipeline
import pandas as pd
from sklearn.model_selection import train_test_split
import torch 
from tensorflow.keras.preprocessing.text import Tokenizer
import string
import pickle
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.bleu_score import SmoothingFunction
from PIL import Image
import requests
from io import BytesIO
import datasets

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cpu'

IMAGE_TO_TEXT ViT-GPT2

In [4]:
image_to_text = pipeline("image-to-text", model="nlpconnect/vit-gpt2-image-captioning", device = -1)

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.
/usr/local/lib/python3.9/dist-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


In [5]:
def get_caption(url):
    try:
        return image_to_text(url)[0]['generated_text']
    except:
        return '#'

CREATIVE CAPTIONS

In [6]:
cc_model = AutoModelWithLMHead.from_pretrained('/content/drive/MyDrive/Information Retrieval Project/saved_models/common_gen_7k_model')

/usr/local/lib/python3.9/dist-packages/transformers/models/auto/modeling_auto.py:1322: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


In [7]:
tokenizer = AutoTokenizer.from_pretrained("t5-base")
special_tokens = ['<SOS>', '<EOS>', 'write creatively: ']
tokenizer.add_tokens(special_tokens, special_tokens=True)

/usr/local/lib/python3.9/dist-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


3

In [15]:
def get_creative_caption(descriptive_caption):
    prefix = 'write creatively: '
    s = '<SOS> ' + descriptive_caption + ' <EOS>'
    s = prefix + s
    s = tokenizer(s, max_length = 47, truncation = True, padding = True, return_tensors = 'pt').input_ids
    s = cc_model.generate(s)
    return tokenizer.batch_decode(s)[0]

HASHTAGS

In [57]:
hashtag_model = AutoModelWithLMHead.from_pretrained("/content/drive/MyDrive/Information Retrieval Project/Instagram worthy captions dataset/t5_hashtag_model_10_epochs_2")

/usr/local/lib/python3.9/dist-packages/transformers/models/auto/modeling_auto.py:1322: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


In [58]:
hashtag_tokenizer = AutoTokenizer.from_pretrained('/content/drive/MyDrive/Information Retrieval Project/Instagram worthy captions dataset/t5_hashtag_model_10_epochs_2')

In [59]:
def gen_hashtags(creative_caption):
    prefix = 'generate tags: '
    temp = prefix + creative_caption
    temp_token = hashtag_tokenizer(temp, max_length = 32, truncation = True, padding = True, return_tensors = 'pt').input_ids
    tags =  hashtag_model.generate(temp_token)
    return hashtag_tokenizer.batch_decode(tags)[0]

In [62]:
tags = gen_hashtags("Seaside tranquility")

In [61]:
tags

'<pad><SOT> #BFFGoal<EOT></s>'

In [ ]:
tags[tags.find('<SOT>') + 5:tags.find('<EOT>')]

EMOJIS

In [30]:
emoji_path = '/content/drive/MyDrive/Information Retrieval Project/saved_models/common_gen_emoji_final_model'
emoji_tokenizer = AutoTokenizer.from_pretrained(emoji_path)
emoji_model = AutoModelWithLMHead.from_pretrained(emoji_path)

/usr/local/lib/python3.9/dist-packages/transformers/models/auto/modeling_auto.py:1322: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


In [31]:
def get_emoji(creative_caption):
    prefix = 'add emojis: '
    temp = prefix + creative_caption
    temp_token = tokenizer(temp, max_length = 39, truncation = True, padding = True, return_tensors = 'pt').input_ids
    tags =  emoji_model.generate(temp_token)
    return emoji_tokenizer.batch_decode(tags)[0]

In [33]:
e = get_emoji('"Friscoeing adventure! A young boy enjoying a frisb #Friends hipGoals #Youthfu lAdventures ')

/usr/local/lib/python3.9/dist-packages/transformers/generation/utils.py:1313: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [36]:
e[e.find('<pad>') + 5: e.find('</s>')]

'🌟 ❤ '

FINAL RUN

In [86]:
# Enter the image_path here
image_path = 'https://bd.gaadicdn.com/processedimages/royal-enfield/classic350/source/classic350612f277880878.jpg'
descriptive_caption = get_caption(image_path)
print("descriptive_caption: " + descriptive_caption)

creative_caption = get_creative_caption(descriptive_caption)
creative_caption = creative_caption[creative_caption.find('<SOS>') + 5:creative_caption.find('<EOS>')]
print("creative_caption: " + creative_caption)

tags = gen_hashtags(creative_caption)
tags = tags[tags.find('<SOT>') + 5:tags.find('<EOT>')]

e = get_emoji(creative_caption)
e = e[e.find('<pad>') + 5: e.find('</s>')]

final_caption = creative_caption+e+tags

print("FINAL: " + final_caption)

/usr/local/lib/python3.9/dist-packages/transformers/generation/utils.py:1313: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


creative_caption:  "A tree with a lot of leaves"<unk> 
FINAL:  "A tree with a lot of leaves"<unk> 🎨  #Beach Vibes #Haped Feelings
